# Exercise 1 - Signing Simply with RSA

This first exercise shows how to prove the validity of a credential using an RSA signature.
Then we create a simple version of Selective Disclosure (SD) to hide some of the data.
Finally we'll fix our errors in the simple SD implementation.

## Sections

1. Basic E-ID example using RSA cryptographic scheme
2. Selective Disclosure using hashing
3. Discussion: security of this scheme, and introduction to unlinkability
4. Coding exercise: protect the hashes

---

## 1. Basic E-ID example using RSA cryptographic scheme

In this first section you'll learn how to do a simple protection of a credentials using signing.
You'll find the following elements:

- credential: an object holding personal data
- issuer: a trusted entity who can sign a credential
- holder: the person described by the personal data
- verifier: wanting to learn parts of the personal data of the holder

### Definition: Verifiable Credentials
A verifiable credential, in its simplest form, is a signed string of data. An issuer will issue a credential by signing a specific string of data then sharing that string of data along with a cryptographic signature that can prove that this string was authorized/ issued by this specific issuer.

In [4]:
// We start by creating a typical E-ID credential object that we will use through out this exercise
const date = new Date("1993-08-01T00:00:00")
const ID_DATA = {
    name: "Jack Sparrow",
    timeOfBirth: date.getTime(),
    profession: "IT Manager"
}

### Issuer

The issuer has a public/private key pair.
We suppose that its public key is known to everybody through an appropriate Public Key Infrastructure (PKI).
If a holder wants to use their E-ID, they first need to get a verified credential from the issuer.
This verified credential is simply the credential + a signature from the issuer.

In [5]:
import * as crypto from 'crypto';

In [6]:
/* 
 For an issuer to be able to start issuing Verifiable Credentials, it first needs
 to have its own cryptographic key pair. 
 Issuers will sign the data using their private key. 
*/
const { publicKey, privateKey } = crypto.generateKeyPairSync('rsa', {
    modulusLength: 4096,
    publicKeyEncoding: {
      type: 'spki',   
    
      format: 'pem'
    },
    privateKeyEncoding: {
      type: 'pkcs8',
      format: 'pem',
    }
});

In [7]:
// An issuer will convert the data object into a string
const message = JSON.stringify(ID_DATA)

// then create a signature for that data
const signer = crypto.createSign('SHA256');
signer.update(message);
const signature = signer.sign(privateKey, 'base64')

console.log(message);
console.log("----------------------------")
console.log(signature);

{"name":"Jack Sparrow","timeOfBirth":744156000000,"profession":"IT Manager"}
----------------------------
phw7rAap7rGCEMqLUo7vU4c5b+Jx/ON38CdsBjzkDqmoLHikpxNiPtLQgsadJV9xu75gCRqWVAtx0SXYm6I94GUwC25h2vY0fIMGhh+dnkyH55QX7UvzZPS6tlG7zs8MQMqrpAVIcK6rRAZzNs6noGeRFfC8RIIGvXPzhOoEh3hZXzJVrgctvciEu4K7VP1DJ4qFfuFsHABdfUtmGKJDnfMfg1ebYq9haedl6cWXQG+IJdNyfv5KVrkF4XeFvy3S7JNh5XPsbsp5rmaNPq6Toq0w0JO7VG8toRHrpYYfShZDLFFbbEukOg317E3xAtTPlbODTlAls/muLPvmn+TcUP/H8lB4/iIbXIvc07Xoa2WnW1HiMxgX5dqIDTZwjkaPS9FwCJO3/Kek2uaKCCpQT/rTy19/L8ZeKNnV4Uc9RNUyzg9hn7hliJrhfboZqvXDuXKBilTLC5XlyN0i9bdWE3Jcs81zfW/af03hSmt7UKaidY2dHNduzaavgi9RchboK/sXMIXJVJWz7P0xcpIo7M4GPBZvQX20+39dsHyYGP4juzeBIbZ0f1tALPuI8WwkhoUz2hAGUat+5BFJSkfYPdjTWFA40JC/jON8B261BuJXa2ZHxLsL29HkRA5n8jCWFRonjpTNI9Q9eDPzW4osMVJvlzF3EOg3oIHlIGkWTKI=


### Issuer →  Holder

After the signature is created, the data along with the signature is transfered to the holder

In this case, the holder can only do one thing with this data which is to share the whole data string along with the signature

### Holder →  Verifier

Once the data is sent from holder to verifier, the verifier can verify that information as follows:

In [27]:
// verifier now recieves "message" and "signature" data
// The verifier should also have access to the publicKey of the issuer
// through a Public Key Infrastructure (PKI)

let verifier = crypto.createVerify('SHA256');
verifier.update(message)
console.log("The signature is correct:", verifier.verify(publicKey, signature, 'base64'))

The signature is correct: true


### Exercise

Make sure that the signature fails in the following cases:

- the message is different from the message used in the signing process
- the public key is different than the public key from the issuer

---

## 2. Selective Disclosure using hashing

What if the holder of the credential wants to only share his name and profession but not his timeOfBirth?
The current implementation wouldn't allow for that, so we will need to change it.
The simplest solution is to hash all fields, and then only send the fields to be disclosed to the verifier.

### Issuer

In [10]:
// One way to implement selective disclosure is to hash every value, and the 
// holder of the credential would share only the values he intends to. 

// issuer will hash the values before signing the data
function hashValue(value: string): string {
  const hash = crypto.createHash('sha256');
  hash.update(value);
  return hash.digest('hex');
}
const ID_DATA_HASHED = {
    name: hashValue(ID_DATA['name']),
    timeOfBirth: hashValue(ID_DATA['timeOfBirth'].toString()),
    profession: hashValue(ID_DATA['profession'])
}

// An issuer will convert the data object into a string
const message = JSON.stringify(ID_DATA_HASHED);

// then create a signature for that data
const signer = crypto.createSign('SHA256');
signer.update(message);

const signature = signer.sign(privateKey, 'base64');
console.log(message);
console.log("----------------------------");
console.log(signature);

{"name":"a5d0dd92108bc55cadfaf56e36a60a7121b059d9e0d09c2d24d853e9d1d14ea7","timeOfBirth":"1f2336f8e0f01df150e470be77b33e4c9b9e758293a9e2944140cdeba976ff45","profession":"c249aac9d89f89626fbcaa519d53e386478a22b775414c324e61289e033f1652"}
----------------------------
G16ekoyYOc2Zcq+HhSNF9C+TmWFkdtg4HeL2pcfvnxzXry2LiAS+j6h33Lkvmb1d+CbYz7PGN+Fuk3hrHDbkCeZPNyf7JRFHCvkzU/ED28BpfZuASnl17MvDNF4zI7WbQENLvUsNnNncQcFAtip5yqUZxYjZ8VKVdi/d3LJ3UExrFtzdGmr0Tx7EPHYixYBsdhf+OC90oortBzDDaBwfyakSSoLSrcQKsEOUU2y+9W5aXchRwui5fTBBfs0Yk/94BJDk1sFdGivooStmaLdis8W+nSyyzupaODZrLmDSXy0p5X+IUTV34qHVKT8KUEXb0QHDion4ZtBL5JDZL5N2HxJ7FZGwMkhYfiQYiUfqcdSWYPwVS2Uu+A/JqYoDxpdAZfZPp9RFXr3xUUWDP5TFvXHWQyr7pCqxkrWCSuRxWxNQgp6syIm53khNwHAt0NG606QrUv3E32Z0ulYN7r+NHo+kbRtL3ygcpVeCLiAM2JD1oMceIOHFzl1mLs0LXCK/PDN6yx9GmVl5/jvWgkPnTl+qQcnH1oPk39dpyqGPSaBtmXnvRXFhyiThID1ovSKjD3/eROwGFtwZOtFQBvaMmosXGkNeN3yWpr6ZWwTmu7e9KD7nJGVbMjvSo4K2zUFyrlUnHL5AMHxgmsifG7+n7Hk2KS+JFxjHz/JRgKrQ1/0=


### Issuer -> Holder

Now the issuer will send the original data, the hashed data, and the sigature to the holder.

### Issuer

If the holder wants to disclose their name and profession, but keep the time of birth private, they will do the followig:

In [14]:
// The holder discloses the value for the name and and profession:
const HOLDER_DISCLOSED_DATA = {
    name: ID_DATA['name'],
    profession: ID_DATA['profession']
}

### Holder -> Verifier

Now the holder will send the following data to the verifier:

- `ID_DATA_HASHED`
- `signature`
- `HOLDER_DISCLOSED_DATA`

This means that the verifier doesn't have access to the `timeOfBirth`.

### Verifier

The verifier now wants to make sure that the data they got from the holder are correct.

In [28]:
// First the verifier has to check that the signature on the hashes is correct:

let verifier = crypto.createVerify('SHA256');
verifier.update(message)
console.log("Signature verification:", verifier.verify(publicKey, signature, 'base64'))

// Now the verifier can compare the disclosed data with the hashed values in the credential.
// If they are equal, and the hash-function is cryptographically secure, the verifier can be covinced that the data is correct.

// 1. Retrieve data object from JSON data
const RETRIEVED_DATA = JSON.parse(message);

// 2. Compare the hashes of the credential with the hashes of the shared data
for (const [key, value] of Object.entries(HOLDER_DISCLOSED_DATA)){
    console.log("Verifying", key, ":", hashValue(value), "==", RETRIEVED_DATA[key], "?");
    if (hashValue(value) != RETRIEVED_DATA[key]) {
        throw new Error(`Reconstructed data in key ${key} is not the same as the hashed counterpart`);
    }
}

// Since we've already verified that the hashed message is valid in the previous code cell
// and now we verified that the hashed values are equal to the revealed values, then
// we conclude that we trust these revealed data.

console.log("Hashed values are equal, so the following is verified:", HOLDER_DISCLOSED_DATA);

Signature verification: true
Verifying name : a5d0dd92108bc55cadfaf56e36a60a7121b059d9e0d09c2d24d853e9d1d14ea7 == a5d0dd92108bc55cadfaf56e36a60a7121b059d9e0d09c2d24d853e9d1d14ea7 ?
Verifying profession : c249aac9d89f89626fbcaa519d53e386478a22b775414c324e61289e033f1652 == c249aac9d89f89626fbcaa519d53e386478a22b775414c324e61289e033f1652 ?
Hashed values are equal, so the following is verified: { name: 'Jack Sparrow', profession: 'IT Manager' }


UncaughtException: Error: Unexpected pending rebuildTimer
    at sys.setTimeout (/Users/ligasser/Programming/C4DT/eid-workshop/node_modules/tslab/dist/converter.js:111:19)
    at Object.scheduleInvalidateResolutionsOfFailedLookupLocations (/Users/ligasser/Programming/C4DT/eid-workshop/node_modules/@tslab/typescript-for-tslab/lib/typescript.js:122719:55)
    at /Users/ligasser/Programming/C4DT/eid-workshop/node_modules/@tslab/typescript-for-tslab/lib/typescript.js:121374:24
    at cb (/Users/ligasser/Programming/C4DT/eid-workshop/node_modules/tslab/dist/converter.js:184:13)
    at /Users/ligasser/Programming/C4DT/eid-workshop/node_modules/@tslab/typescript-for-tslab/lib/typescript.js:5796:9
    at /Users/ligasser/Programming/C4DT/eid-workshop/node_modules/@tslab/typescript-for-tslab/lib/typescript.js:5560:101
    at Array.forEach (<anonymous>)
    at /Users/ligasser/Programming/C4DT/eid-workshop/node_modules/@tslab/typescript-for-tslab/lib/typescript.js:5560:85
    at FSWatcher.callback

### Exercise

- Print the hashes of `hashValue(value)` and from the `RETRIEVED_DATA(key)` and compare them visually
- Change the disclosed fields and make sure it still rus

---

## 3. Discussion: security of this scheme, and introduction to unlinkability

### Security of hashed values

To keep some anonymity, the fields which the holder decides not to disclose remain hashed.
How secure is this?
For example, if the holder selectively discloses the `profession`, what can the verifier do with the other fields?

### Unlinkability

One of the big problems in current day ads is that even if you visit different websites, the ad-industry will correlate these visits into a single user profile.
This allows these data brokers to sell your profile not only for ads, but also for influence campains, and for geo-tracking.
Not only ad companies can do this, but these profiles are sold by the data brokers also to the government, or even to private persons!
For this reason, presenting a credential multiple times should be **unlinkable**.

Does the current scheme guarantee unlinkability of the holder towards different verifiers?

---

## 4. Coding exercise: protect the hashes

Right now, the `timeOfBirth` can be guessed by looping over all possible dates to figure out someone's exact date of birth.

- Create a way to hack the `timeOfBirth`. How can this be made faster?
- How would you hack the `name`, or `profession`?
- Reimplement the communication between the holder and verifier modifying the hash function in a way that doesn't make it easy to guess the fields